In [1]:
%%capture
%pip install torchvision
%pip install torchsummary
%pip install torchviz
%pip install hiddenlayer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/Othercomputers/My Computer/Masters_Staff/trimester_3/Deep_Learning

Mounted at /content/drive
/content/drive/Othercomputers/My Computer/Masters_Staff/trimester_3/Deep_Learning


In [3]:
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchsummary import summary
from torch.autograd import Variable
from torchvision import datasets, transforms

from torchvision.utils import save_image

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 20 14:07:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [24]:
#model params
params = {
    'batch_size':32,
    'latent_dim': 100,
    'mul': 100,
    'num_classes':10,
    'channels':3,
    'img_size':32,
    'device':torch.device('cuda') if torch.cuda.is_available() else 'cpu',
    'fliplr':True,
    'num_epochs':50,
    'decay_epoch':50,
    'lr':0.0002,    #learning rate for generator
    'b1':0.5 ,    #beta1 for Adam optimizer
    'b2':0.999 ,  #beta2 for Adam optimizer
    'sample_interval':400,
    'n_cpu':8
}

cuda = True if torch.cuda.is_available() else False

In [25]:
# Function to load and preprocess datasets
def load_dataset(dataset_cls, split, root, download, normalize=False, subset_size=None, batch_size=params['batch_size']):
    if dataset_cls == datasets.MNIST:
        dataset = dataset_cls(root=root, train=(split=='train'), download=download, transform=transforms.ToTensor())
    else:
        dataset = dataset_cls(root=root, split=split, download=download, transform=transforms.ToTensor())

    if normalize:
        # mean, std = compute_mean_and_std(dataset)

        if dataset_cls == datasets.MNIST:

            print(f"Dataset is MNIST: {dataset_cls}")
            transform = transforms.Compose([
                transforms.Resize(( params['img_size'],params['img_size']) ),
                transforms.Grayscale(params['channels']),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5]*params['channels'], std=[0.5]*params['channels'])
            ])
        else:
            print(f"Dataset is SVHN: {dataset_cls}")
            transform = transforms.Compose([
                transforms.Resize(( params['img_size'],params['img_size'] )),
                transforms.Grayscale(params['channels']),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5]*params['channels'], std=[0.5]*params['channels'])
            ])

        # Apply the computed normalization
        dataset.transform = transform

    if subset_size:
        indices = np.random.choice(len(dataset), subset_size, replace=False)
        print(dataset.transform)
        dataset = Subset(dataset, indices)

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True if split == 'train' else False, num_workers=1)
    return loader

# Load MNIST datasets
mnist_trainloader = load_dataset(datasets.MNIST, 'train', './data/', True, normalize=True,subset_size=params['batch_size']*params['mul'] )
mnist_testloader = load_dataset(datasets.MNIST, 'test', './data/', True, normalize=True)

# Load SVHN datasets
svhn_trainloader = load_dataset(datasets.SVHN, 'train', './data/', True, normalize=True,subset_size=params['batch_size']*params['mul'])
svhn_testloader = load_dataset(datasets.SVHN, 'test', './data/', True, normalize=True)

# Optionally, load the extra SVHN dataset if needed
# svhn_extraloader = load_dataset(datasets.SVHN, 'extra', './data/', True, normalize=True, subset_size=20000)

# Verify the sizes of the datasets
print(f'MNIST train subset size: {len(mnist_trainloader.dataset)}')
# print(f'SVHN train subset size: {len(svhn_trainloader.dataset)}')
# print(f'SVHN extra size: {len(svhn_extraloader.dataset)}')



Dataset is MNIST: <class 'torchvision.datasets.mnist.MNIST'>
Compose(
    Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=3)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)
Dataset is MNIST: <class 'torchvision.datasets.mnist.MNIST'>
Using downloaded and verified file: ./data/train_32x32.mat
Dataset is SVHN: <class 'torchvision.datasets.svhn.SVHN'>
Compose(
    Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=3)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)
Using downloaded and verified file: ./data/test_32x32.mat
Dataset is SVHN: <class 'torchvision.datasets.svhn.SVHN'>
MNIST train subset size: 3200


In [7]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transformations (same as during training)
transform = transforms.Compose([
    transforms.Grayscale(3),  # Ensure images are single-channel grayscale
    transforms.ToTensor(),
    transforms.Normalize([0.5]*params['channels'], [0.5]*params['channels'])  # Assuming normalization parameters used during training
])

# Define the path to the transformed images
transformed_images_path = 'transformed_mnist_train'

# Create the dataset
transformed_dataset = datasets.ImageFolder(root=transformed_images_path, transform=transform)

# Create the DataLoader
transformed_dataloader = DataLoader(transformed_dataset, batch_size=params['batch_size'], shuffle=True, num_workers=2)

In [8]:
# Define weight initialization function
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Linear") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

# Define Coupled Generators
class CoupledGenerators(nn.Module):
    def __init__(self):
        super(CoupledGenerators, self).__init__()

        self.init_size = params['img_size'] // 4
        self.fc = nn.Sequential(nn.Linear(params['latent_dim'], 128 * self.init_size ** 2))

        self.shared_conv = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
        )
        self.G1 = nn.Sequential(
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, params['channels'], 3, stride=1, padding=1),
            nn.Tanh(),
        )
        self.G2 = nn.Sequential(
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, params['channels'], 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.fc(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img_emb = self.shared_conv(out)
        features1 = self.G1[:-1](img_emb)  # Get features before the final Tanh layer
        features2 = self.G2[:-1](img_emb)  # Get features before the final Tanh layer
        img1 = self.G1[-1](features1)
        img2 = self.G2[-1](features2)
        return img1, img2, features1, features2

# Define the classifier with the correct input dimension
class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

# Define Coupled Discriminators
class CoupledDiscriminators(nn.Module):
    def __init__(self):
        super(CoupledDiscriminators, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            block.extend([nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)])
            return block

        self.shared_conv = nn.Sequential(
            *discriminator_block(params['channels'], 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )
        # The height and width of downsampled image
        ds_size = params['img_size'] // 2 ** 4
        self.D1 = nn.Linear(128 * ds_size ** 2, 1)
        self.D2 = nn.Linear(128 * ds_size ** 2, 1)

    def forward(self, img1, img2):
        # Determine validity of first image
        out = self.shared_conv(img1)
        out = out.view(out.shape[0], -1)
        validity1 = self.D1(out)
        # Determine validity of second image
        out = self.shared_conv(img2)
        out = out.view(out.shape[0], -1)
        validity2 = self.D2(out)

        return validity1, validity2

# class CoupledGenerators(nn.Module):
#     def __init__(self):
#         super(CoupledGenerators, self).__init__()

#         self.init_size = params['img_size'] // 4
#         self.fc = nn.Sequential(nn.Linear(params['latent_dim'], 128 * self.init_size ** 2))

#         self.shared_conv = nn.Sequential(
#             nn.BatchNorm2d(128),
#             nn.Upsample(scale_factor=2),
#             nn.Conv2d(128, 128, 3, stride=1, padding=1),
#             nn.BatchNorm2d(128, 0.8),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Upsample(scale_factor=2),
#         )
#         self.G1 = nn.Sequential(
#             nn.Conv2d(128, 64, 3, stride=1, padding=1),
#             nn.BatchNorm2d(64, 0.8),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv2d(64, params['channels'], 3, stride=1, padding=1),
#             nn.Tanh(),
#         )
#         self.G2 = nn.Sequential(
#             nn.Conv2d(128, 64, 3, stride=1, padding=1),
#             nn.BatchNorm2d(64, 0.8),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv2d(64, params['channels'], 3, stride=1, padding=1),
#             nn.Tanh(),
#         )

#     def forward(self, noise):
#         out = self.fc(noise)
#         out = out.view(out.shape[0], 128, self.init_size, self.init_size)
#         img_emb = self.shared_conv(out)
#         img1 = self.G1(img_emb)
#         img2 = self.G2(img_emb)
#         return img1, img2




In [9]:
# Loss function
adversarial_loss = torch.nn.MSELoss()

# Initialize models
coupled_generators = CoupledGenerators()
coupled_discriminators = CoupledDiscriminators()

input_dim = 3 * 32 *32  # Adjust this based on actual size
num_classes = 10  # Assuming 10 classes for digits
classifier = Classifier(input_dim, num_classes)
if cuda:
    classifier.cuda()


if cuda:
    coupled_generators.cuda()
    coupled_discriminators.cuda()
    classifier.cuda()

# Initialize weights
coupled_generators.apply(weights_init_normal)
coupled_discriminators.apply(weights_init_normal)

CoupledDiscriminators(
  (shared_conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Dropout2d(p=0.25, inplace=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.2, inplace=True)
    (10): Dropout2d(p=0.25, inplace=False)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (13): LeakyReLU(negative_slope=0.2, inplace=True)
    (14): Dropout2d(p=0.25, inplace=False)
 

In [10]:
# Optimizers
optimizer_G = torch.optim.Adam(coupled_generators.parameters(), lr=params['lr'], betas=(params['b1'], params['b2']))
optimizer_D = torch.optim.SGD(coupled_discriminators.parameters(), lr=params['lr'])
optimizer_C = torch.optim.Adam(classifier.parameters(), lr=params['lr'], betas=(params['b1'], params['b2']))


Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


In [11]:
# ----------
#  Training
# ----------

dataloader1 = mnist_trainloader
dataloader2 = svhn_trainloader

# Training
EPOCHS = 40
for epoch in range(EPOCHS):
    for i, ((imgs1, labels1), (imgs2, _)) in enumerate(zip(transformed_dataloader, svhn_trainloader)):

        batch_size = imgs1.shape[0]

        # Adversarial ground truths
        valid = Variable(Tensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        imgs1 = Variable(imgs1.type(Tensor).expand(imgs1.size(0), 3, params['img_size'], params['img_size']))
        imgs2 = Variable(imgs2.type(Tensor))

        # ------------------
        # Train Generators
        # ------------------
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (batch_size, params['latent_dim']))))

        # Generate a batch of images
        gen_imgs1, gen_imgs2, features1, features2 = coupled_generators(z)
        # Determine validity of generated images
        validity1, validity2 = coupled_discriminators(gen_imgs1, gen_imgs2)

        g_loss = (adversarial_loss(validity1, valid) + adversarial_loss(validity2, valid)) / 2

        g_loss.backward()
        optimizer_G.step()

        # ------------------
        # Train Classifier
        # ------------------
        optimizer_C.zero_grad()

        # Extract features from the generator for source domain
        features1_c = features1.view(features1.size(0), -1)  # Flatten features

        # Train classifier on source domain features
        labels1 = Variable(labels1.type(torch.LongTensor).cuda())
        outputs = classifier(features1_c)
        c_loss = F.cross_entropy(outputs, labels1)

        c_loss.backward()
        optimizer_C.step()


        # ------------------
        # Train Discriminators
        # ------------------
        optimizer_D.zero_grad()

        # Determine validity of real and generated images
        validity1_real, validity2_real = coupled_discriminators(imgs1, imgs2)
        validity1_fake, validity2_fake = coupled_discriminators(gen_imgs1.detach(), gen_imgs2.detach())

        d_loss = (
            adversarial_loss(validity1_real, valid)
            + adversarial_loss(validity1_fake, fake)
            + adversarial_loss(validity2_real, valid)
            + adversarial_loss(validity2_fake, fake)
        ) / 4

        d_loss.backward()
        optimizer_D.step()



        print(
            f"[Epoch {epoch}/{EPOCHS}] [Batch {i}/{len(mnist_trainloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}] [C loss: {c_loss.item()}]"
        )

        batches_done = epoch * len(mnist_trainloader) + i
        if batches_done % 100 == 0:
            gen_imgs = torch.cat((gen_imgs1.data, gen_imgs2.data), 0)
            save_image(gen_imgs, f"coGAN_img/{batches_done}_.png", nrow=8, normalize=True)

<ipython-input-11-e6b88fc41d01>:16: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  valid = Variable(Tensor(batch_size, 1).fill_(1.0), requires_grad=False)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets


# Define the Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 3*32*32),
            nn.Tanh()
        )

    def forward(self, x):
        return self.fc(x).view(-1, 3, 32, 32)

# Define the shared part of the Discriminator
class SharedDiscriminator(nn.Module):
    def __init__(self):
        super(SharedDiscriminator, self).__init__()
        self.shared_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.shared_layers(x)

# Define the Discriminator with domain-specific and classification heads
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.shared = SharedDiscriminator()
        self.domain_head = nn.Sequential(
            nn.Linear(128 * 8 * 8, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )
        self.classifier_head = nn.Sequential(
            nn.Linear(128 * 8 * 8, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        shared_features = self.shared(x).view(-1, 128 * 8 * 8)
        domain_output = self.domain_head(shared_features)
        classification_output = self.classifier_head(shared_features)
        return domain_output, classification_output
# Initialize the models
G_A = Generator()
G_B = Generator()
D_A = Discriminator()
D_B = Discriminator()

# Loss functions
adversarial_loss = nn.BCELoss()
classification_loss = nn.NLLLoss()

# Optimizers
optimizer_G = optim.Adam(list(G_A.parameters()) + list(G_B.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = optim.SGD(D_A.parameters(), lr=0.00002)
optimizer_D_B = optim.SGD(D_B.parameters(), lr=0.00002)

n_epoch = 20
# Training loop
for epoch in range(n_epoch):
    for (mnist_imgs, mnist_labels), (svhn_imgs, _) in zip(mnist_trainloader, svhn_trainloader):
        batch_size = mnist_imgs.size(0)

        # Adversarial ground truths
        valid = torch.ones(batch_size, 1)
        fake = torch.zeros(batch_size, 1)

        # ---------------------
        #  Train Generators
        # ---------------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(batch_size, 100)

        # Generate a batch of images
        gen_mnist = G_A(z)
        gen_svhn = G_B(z)

        # Loss measures generator's ability to fool the discriminator
        validity_A, _ = D_A(gen_mnist)
        validity_B, _ = D_B(gen_svhn)
        g_loss = adversarial_loss(validity_A, valid) + adversarial_loss(validity_B, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminators
        # ---------------------

        optimizer_D_A.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_pred, _ = D_A(mnist_imgs)
        fake_pred, _ = D_A(gen_mnist.detach())
        d_loss_A = adversarial_loss(real_pred, valid) + adversarial_loss(fake_pred, fake)

        d_loss_A.backward()
        optimizer_D_A.step()

        optimizer_D_B.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_pred, _ = D_B(svhn_imgs)
        fake_pred, _ = D_B(gen_svhn.detach())
        d_loss_B = adversarial_loss(real_pred, valid) + adversarial_loss(fake_pred, fake)

        d_loss_B.backward()
        optimizer_D_B.step()

        # ---------------------
        #  Train Classifiers
        # ---------------------

        _, mnist_class_pred = D_A(mnist_imgs)
        c_loss = classification_loss(mnist_class_pred, mnist_labels)

        c_loss.backward()
        optimizer_D_A.step()

    print(f"Epoch {epoch}/{n_epoch} - G Loss: {g_loss.item()} - D_A Loss: {d_loss_A.item()} - D_B Loss: {d_loss_B.item()} - C Loss: {c_loss.item()}")

# Ensure the model is in evaluation mode
D_B.eval()

with torch.no_grad():
    # Evaluate on SVHN test set
    for svhn_imgs, svhn_labels in svhn_testloader:
        svhn_imgs, svhn_labels = svhn_imgs.to(DEVICE), svhn_labels.to(DEVICE)
        _, svhn_class_pred = D_B(svhn_imgs)
        _, predicted_svhn = torch.max(svhn_class_pred, 1)
        total_svhn += svhn_labels.size(0)
        correct_svhn += (predicted_svhn == svhn_labels).sum().item()

    # Evaluate on MNIST test set
    for mnist_imgs, mnist_labels in mnist_testloader:
        mnist_imgs, mnist_labels = mnist_imgs.to(DEVICE), mnist_labels.to(DEVICE)
        _, mnist_class_pred = D_B(mnist_imgs)
        _, predicted_mnist = torch.max(mnist_class_pred, 1)
        total_mnist += mnist_labels.size(0)
        correct_mnist += (predicted_mnist == mnist_labels).sum().item()

# Calculate accuracy
accuracy_svhn = 100 * correct_svhn / total_svhn
accuracy_mnist = 100 * correct_mnist / total_mnist

# Print results
print(f'SVHN Test Accuracy: {accuracy_svhn:.2f}% ({correct_svhn}/{total_svhn})')
print(f'MNIST Test Accuracy: {accuracy_mnist:.2f}% ({correct_mnist}/{total_mnist})')



In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define weight initialization function
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Linear") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

params = {
    'img_size': 32,
    'latent_dim': 100,
    'channels': 3
}

# Define Coupled Generators
class CoupledGenerators(nn.Module):
    def __init__(self):
        super(CoupledGenerators, self).__init__()

        self.init_size = params['img_size'] // 4
        self.fc = nn.Sequential(nn.Linear(params['latent_dim'], 128 * self.init_size ** 2))

        self.shared_conv = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
        )
        self.G1 = nn.Sequential(
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, params['channels'], 3, stride=1, padding=1),
            nn.Dropout(0.3),
            nn.Tanh(),
        )
        self.G2 = nn.Sequential(
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, params['channels'], 3, stride=1, padding=1),
            nn.Dropout(0.3),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.fc(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img_emb = self.shared_conv(out)
        features1 = self.G1[:-1](img_emb)  # Get features before the final Tanh layer
        features2 = self.G2[:-1](img_emb)  # Get features before the final Tanh layer
        img1 = self.G1[-1](features1)
        img2 = self.G2[-1](features2)
        return img1, img2, features1, features2

# Define the classifier with the correct input dimension
class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

# Define Coupled Discriminators
class CoupledDiscriminators(nn.Module):
    def __init__(self):
        super(CoupledDiscriminators, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            block.extend([nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)])
            return block

        self.shared_conv = nn.Sequential(
            *discriminator_block(params['channels'], 64, bn=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
        )
        # The height and width of downsampled image
        ds_size = params['img_size'] // 2 ** 4
        self.D1 = nn.Sequential(
            nn.Linear(512 * ds_size ** 2, 1),
            nn.Sigmoid()  # Add sigmoid activation
        )
        self.D2 = nn.Sequential(
            nn.Linear(512 * ds_size ** 2, 1),
            nn.Sigmoid()  # Add sigmoid activation
        )
        self.C1 = nn.Linear(512 * ds_size ** 2, 10)
        self.C2 = nn.Linear(512 * ds_size ** 2, 10)

    def forward(self, img1, img2):
        # Determine validity of first image
        out1 = self.shared_conv(img1)
        out1 = out1.view(out1.shape[0], -1)
        validity1 = self.D1(out1)
        class1 = self.C1(out1)

        # Determine validity of second image
        out2 = self.shared_conv(img2)
        out2 = out2.view(out2.shape[0], -1)
        validity2 = self.D2(out2)
        class2 = self.C2(out2)

        return validity1, validity2, class1, class2

# Initialize the models
G = CoupledGenerators()
D = CoupledDiscriminators()

# Apply weight initialization
G.apply(weights_init_normal)
D.apply(weights_init_normal)

# Loss functions
adversarial_loss = nn.BCELoss()
classification_loss = nn.CrossEntropyLoss()

# Optimizers
optimizer_G = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.SGD(D.parameters(), lr=0.0002)


# Training loop
for epoch in range(10):
    for (mnist_imgs, mnist_labels), (svhn_imgs, _) in zip(mnist_trainloader, svhn_trainloader):
        batch_size = mnist_imgs.size(0)

        # Adversarial ground truths
        valid = torch.ones(batch_size, 1)
        fake = torch.zeros(batch_size, 1)

        # ---------------------
        #  Train Generators
        # ---------------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(batch_size, params['latent_dim'])

        # Generate a batch of images
        gen_mnist, gen_svhn, features_mnist, features_svhn = G(z)

        # Loss measures generator's ability to fool the discriminator
        validity_mnist, validity_svhn, _, _ = D(gen_mnist, gen_svhn)
        g_loss = adversarial_loss(validity_mnist, valid) + adversarial_loss(validity_svhn, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminators
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_pred_mnist, real_pred_svhn, class_pred_mnist, _ = D(mnist_imgs, svhn_imgs)
        fake_pred_mnist, fake_pred_svhn, _, _ = D(gen_mnist.detach(), gen_svhn.detach())

        d_loss_real = adversarial_loss(real_pred_mnist, valid) + adversarial_loss(real_pred_svhn, valid)
        d_loss_fake = adversarial_loss(fake_pred_mnist, fake) + adversarial_loss(fake_pred_svhn, fake)
        d_loss = (d_loss_real + d_loss_fake) / 2

        d_loss.backward()
        optimizer_D.step()

        # ---------------------
        #  Train Classifiers
        # ---------------------

        optimizer_D.zero_grad()

        class_pred_mnist, _ = D(mnist_imgs, svhn_imgs)[2:]
        c_loss = classification_loss(class_pred_mnist, mnist_labels)

        c_loss.backward()
        optimizer_D.step()

    print(f"Epoch {epoch}/{10} - G Loss: {g_loss.item()} - D Loss: {d_loss.item()} - C Loss: {c_loss.item()}")

# Initialize variables for tracking accuracy
correct_svhn = 0
total_svhn = 0
correct_mnist = 0
total_mnist = 0

# Ensure the model is in evaluation mode
D.eval()

with torch.no_grad():
    # Evaluate on SVHN test set
    for svhn_imgs, svhn_labels in svhn_testloader:
        svhn_imgs, svhn_labels = svhn_imgs.to(DEVICE), svhn_labels.to(DEVICE)
        _, svhn_class_pred = D(svhn_imgs,torch.zeros_like(svhn_imgs))
        _, predicted_svhn = torch.max(svhn_class_pred, 1)
        total_svhn += svhn_labels.size(0)
        correct_svhn += (predicted_svhn == svhn_labels).sum().item()

    # Evaluate on MNIST test set
    for mnist_imgs, mnist_labels in mnist_testloader:
        mnist_imgs, mnist_labels = mnist_imgs.to(DEVICE), mnist_labels.to(DEVICE)
        _, mnist_class_pred = D(mnist_imgs,torch.zeros_like(mnist_imgs))
        _, predicted_mnist = torch.max(mnist_class_pred, 1)
        total_mnist += mnist_labels.size(0)
        correct_mnist += (predicted_mnist == mnist_labels).sum().item()

# Calculate accuracy
accuracy_svhn = 100 * correct_svhn / total_svhn
accuracy_mnist = 100 * correct_mnist / total_mnist

# Print results
print(f'SVHN Test Accuracy: {accuracy_svhn:.2f}% ({correct_svhn}/{total_svhn})')
print(f'MNIST Test Accuracy: {accuracy_mnist:.2f}% ({correct_mnist}/{total_mnist})')



Epoch 0/10 - G Loss: 1.2886850833892822 - D Loss: 1.4369761943817139 - C Loss: 2.3049843311309814
Epoch 1/10 - G Loss: 1.2301700115203857 - D Loss: 1.4619617462158203 - C Loss: 2.293734312057495
Epoch 2/10 - G Loss: 1.1831457614898682 - D Loss: 1.4919672012329102 - C Loss: 2.3105039596557617
Epoch 3/10 - G Loss: 1.1616599559783936 - D Loss: 1.5028189420700073 - C Loss: 2.3045589923858643
Epoch 4/10 - G Loss: 1.1548199653625488 - D Loss: 1.5147151947021484 - C Loss: 2.30519700050354
Epoch 5/10 - G Loss: 1.1470779180526733 - D Loss: 1.515200138092041 - C Loss: 2.296841859817505
Epoch 6/10 - G Loss: 1.1428532600402832 - D Loss: 1.5149661302566528 - C Loss: 2.288012981414795
Epoch 7/10 - G Loss: 1.1464552879333496 - D Loss: 1.5161023139953613 - C Loss: 2.2940988540649414
Epoch 8/10 - G Loss: 1.1589207649230957 - D Loss: 1.506476640701294 - C Loss: 2.2968218326568604
Epoch 9/10 - G Loss: 1.1611943244934082 - D Loss: 1.5087745189666748 - C Loss: 2.305819511413574


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same